In [179]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import re, string
from gensim.parsing.preprocessing import remove_stopwords

In [156]:
dataFake = pd.read_csv("Fake.csv")
dataTrue = pd.read_csv("True.csv")

In [157]:
dataFake["class"]=0
dataTrue["class"]=1

In [158]:
# Manual test
dataFake_manualTest = dataFake.tail(10)
dataTrue_manualTest = dataTrue.tail(10)
for i in range(23480,23470,-1):
  dataFake.drop([i],axis=0,inplace=True)

for i in range(21416,21406,-1):
  dataTrue.drop([i],axis=0,inplace=True)

In [ ]:
dataFake_manualTest["class"]=0
dataTrue_manualTest["class"]=1

In [ ]:
print(dataFake.shape)
print(dataTrue.shape)
print(dataFake_manualTest.shape)
print(dataTrue_manualTest.shape)

In [161]:
data_merge = pd.concat([dataFake,dataTrue], axis=0)

In [162]:
data = data_merge.drop(["title","subject","date"], axis=1)

In [ ]:
data.head()

In [164]:
def word_cleaner(text):
  text = text. lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('\\W', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return remove_stopwords(text)  

In [165]:
data["text"]= data['text'].apply(word_cleaner)

In [166]:
x = data['text']
y = data['class']

In [167]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25)

In [168]:
vector = TfidfVectorizer()
xv_train = vector.fit_transform(x_train)
xv_test = vector.transform(x_test)

In [169]:
LR = LogisticRegression()

In [170]:
LR.fit(xv_train,y_train)

LogisticRegression()

In [ ]:
predicts = LR.predict(xv_test)
LR.score(xv_test,y_test)
print(classification_report(y_test,predicts))

In [172]:
def output(n):
  if n==0:
    return "Fake New"
  elif n==1:
    return "True New"

def test(text):
  test_new = {"text":[text]}
  new_def = pd.DataFrame(test_new)
  new_def["text"] =new_def["text"].apply(word_cleaner)
  x_test = new_def["text"]
  x_test = vector.transform(x_test)
  pred = LR.predict(x_test)
  return output(pred[0])

In [173]:
new = """Breaking News: Scientists Discover Ancient Civilization Underneath the Antarctic Ice

In a stunning archaeological breakthrough, a team of international scientists has uncovered the remains of an ancient civilization buried deep beneath the icy plains of Antarctica. This groundbreaking discovery is set to challenge our understanding of human history and raise intriguing questions about the origins of civilization.

The expedition, led by renowned archaeologist Dr. Elizabeth Morgan, embarked on an ambitious mission to explore the uncharted regions of Antarctica. Using cutting-edge satellite imaging technology, they identified a peculiar pattern of structures buried beneath layers of ice, hinting at the presence of a long-lost civilization.

As the team carefully excavated the site, they uncovered a vast network of interconnected structures, resembling an advanced city. The architecture displayed remarkable precision and intricate designs, suggesting a highly skilled and technologically advanced society. The scientists were astounded by the sheer scale and complexity of the discovered structures.

Further exploration of the ancient city revealed evidence of a sophisticated infrastructure system, including a complex network of underground tunnels. These tunnels, believed to have served various purposes such as transportation and communication, were a testament to the advanced engineering capabilities of this enigmatic civilization.

The artifacts recovered from the excavation site provide intriguing insights into the lives of the ancient Antarctic inhabitants. Intricately carved statues, pottery adorned with intricate patterns, and remnants of advanced tools and machinery were among the findings. The level of craftsmanship displayed by these artifacts was unparalleled, hinting at a civilization far ahead of its time.

Dr. Morgan and her team have tentatively dated the civilization to be over 10,000 years old, predating many known ancient civilizations by several millennia. This discovery challenges the conventional theories of human history and raises questions about the possibility of early transoceanic travel and cultural diffusion.

The implications of this remarkable discovery extend beyond our understanding of human history. The advanced technologies and architectural achievements of this civilization may hold valuable lessons for our modern world. Scientists and historians are already speculating on the potential impact this finding could have on fields such as sustainable urban planning, engineering, and even space exploration.

While the news of this extraordinary discovery has sparked excitement among the scientific community and the general public, experts urge caution and further investigation. Skeptics argue that the evidence presented needs to undergo rigorous scrutiny and peer review before definitive conclusions can be drawn.

As the world eagerly awaits more information about this ancient Antarctic civilization, the team of researchers plans to continue their excavations and analysis. The findings have the potential to rewrite history books and provide new insights into the development of human civilization on Earth.

Stay tuned for updates on this groundbreaking discovery as we delve deeper into the mysteries of this forgotten civilization beneath the icy continent of Antarctica."""

In [ ]:
print(test(new))